In [3]:
import pandas as pd
import numpy as np
import re
import warnings
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import joblib # Thêm thư viện joblib

warnings.filterwarnings('ignore')

# ===================================================================
# BƯỚC 1: TẢI DỮ LIỆU
# ===================================================================

train_df = pd.read_csv('dataset/train.csv')
val_df = pd.read_csv('dataset/val.csv')
test_df = pd.read_csv('dataset/test.csv')

# Loại bỏ các cột không cần thiết
cols_to_drop = ['user_name', 'timestamp_post', 'num_like_post', 'num_comment_post', 'num_share_post']
train_df.drop(cols_to_drop, axis=1, inplace=True)
val_df.drop(cols_to_drop, axis=1, inplace=True)
test_df.drop(cols_to_drop, axis=1, inplace=True)


# ===================================================================
# BƯỚC 2: HÀM TIỀN XỬ LÝ (PREPROCESSING FUNCTIONS)
# ===================================================================

def remove_emoji(text):
    """Loại bỏ emoji"""
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def clean_text(text):
    """Làm sạch văn bản tiếng Việt"""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = remove_emoji(text)
    text = re.sub(r'http\S+|www\S+|https\S+|<url>', '', text, flags=re.MULTILINE) # URL
    text = re.sub(r'\S+@\S+', '', text) # Email
    text = re.sub(r'\b\d{10,11}\b', '', text) # Số điện thoại
    text = re.sub(r'<.*?>', '', text) # HTML
    text = text.replace('_', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^a-zA-Z0-9àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ\s.,!?_-]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def vietnamese_tokenize(text):
    """Tách từ tiếng Việt"""
    if not text:
        return ""
    try:
        tokenized_text = word_tokenize(text, format="text")
        return tokenized_text
    except:
        return text

VIETNAMESE_STOPWORDS= set([
    'bị', 'bởi', 'cả', 'các', 'cái', 'cần', 'càng', 'chỉ', 'chiếc',
    'cho', 'chứ', 'chưa', 'chuyện', 'có', 'có_thể', 'cứ', 'của',
    'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đến_nỗi', 'đều',
    'điều', 'do', 'đó', 'được', 'dưới', 'gì', 'khi', 'không',
    'là', 'lại', 'lên', 'lúc', 'mà', 'mỗi', 'một_cách', 'này',
    'nên', 'nếu', 'ngay', 'nhiều', 'như', 'nhưng', 'những', 'nơi',
    'nữa', 'phải', 'qua', 'ra', 'rằng', 'rất', 'rồi', 'sau',
    'sẽ', 'so', 'sự', 'tại', 'theo', 'thì', 'trên', 'trước',
    'từ', 'từng', 'và', 'vẫn', 'vào', 'vậy', 'vì', 'việc',
    'với', 'vừa', 'ai', 'anh', 'bao_giờ', 'bao_lâu', 'bao_nhiêu', 'bên', 'bộ',
    'chị', 'chúng_ta', 'chúng_tôi', 'cuộc', 'em', 'hết', 'họ',
    'hoặc', 'khác', 'kể', 'khiến', 'làm', 'loại', 'lòng', 'mình',
    'muốn', 'người', 'nhà', 'nhất', 'nhỏ', 'những', 'năm', 'nào',
    'này', 'nào', 'nếu', 'ông', 'qua', 'quá', 'quyển', 'sau_đó',
    'thằng', 'thì', 'thứ', 'tin', 'tôi', 'tới', 'vài', 'vẫn',
    'về', 'việc', 'vòng', 'xa', 'xuống', 'ý', 'đã', 'đem', 'đến',
    'định', 'đó', 'đời', 'đồng_thời', 'để', 'đều', 'đi', 'điều',
    'đơn_vị', 'được', 'gần', 'họ', 'giờ', 'hay', 'hơn', 'ít',
    'liên_quan', 'lúc', 'lên', 'mấy', 'ngoài', 'nhiều', 'nhằm',
    'như_vậy', 'phía', 'trong', 'tuy', 'từng', 'tới', 'về',
    'với', 'xem'
])

def remove_stopwords(text, stopwords=VIETNAMESE_STOPWORDS):
    """Loại bỏ stopwords"""
    if not text: return ""
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

def normalize_repeated_chars(text):
    return re.sub(r'(.)\1{2,}', r'\1', text)

def remove_extra_punctuation(text):
    text = re.sub(r'[.]{2,}', '.', text)
    text = re.sub(r'[!]{2,}', '!', text)
    text = re.sub(r'[?]{2,}', '?', text)
    return text

def normalize_numbers(text):
    return text

def preprocess_pipeline(text, remove_stop=True):
    text = clean_text(text)
    text = normalize_repeated_chars(text)
    text = remove_extra_punctuation(text)
    text = vietnamese_tokenize(text)
    if remove_stop:
        text = remove_stopwords(text)
    text = normalize_numbers(text)
    return text

def preprocess_dataframe(df, text_column='post_message', remove_stop=True):
    print(f"Đang xử lý {len(df)} mẫu...")
    df['cleaned_text'] = df[text_column].apply(
        lambda x: preprocess_pipeline(x, remove_stop=remove_stop)
    )
    df = df[df['cleaned_text'].str.strip() != '']
    print(f"Hoàn thành! Còn lại {len(df)} mẫu sau khi xử lý.")
    return df


# ===================================================================
# BƯỚC 3: ÁP DỤNG TIỀN XỬ LÝ
# ===================================================================

train_processed = preprocess_dataframe(train_df, remove_stop=False)
test_processed = preprocess_dataframe(test_df, remove_stop=False)

X_train = train_processed['cleaned_text']
y_train = train_processed['label']
X_test = test_processed['cleaned_text']


# ===================================================================
# BƯỚC 4: TẠO FEATURE (TF-IDF)
# ===================================================================

print("\nĐang tạo TF-IDF Features...")
vectorizer = TfidfVectorizer(max_features=50000,
                             ngram_range=(1, 2))

X_train_vectorized = vectorizer.fit_transform(X_train)

# --- THÊM BƯỚC LƯU VECTORIZER ---
print("Đang lưu TF-IDF Vectorizer vào 'tfidf_vectorizer.pkl'...")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
print("Lưu Vectorizer hoàn tất.")
# --------------------------------

X_test_vectorized = vectorizer.transform(X_test)


# ===================================================================
# BƯỚC 5: XÂY DỰNG VÀ HUẤN LUYỆN MÔ HÌNH
# ===================================================================

## Mô hình 1: Logistic Regression
print("\n--- Mô hình 1: Logistic Regression ---")
logreg_model = LogisticRegression(
    max_iter=1000,
    class_weight='balanced'
)
print("Đang huấn luyện Logistic Regression...")
logreg_model.fit(X_train_vectorized, y_train)
print("Logistic Regression huấn luyện hoàn tất.")

# --- THÊM BƯỚC LƯU MÔ HÌNH 1 ---
print("Đang lưu mô hình Logistic Regression vào 'logreg_fake_news_model.pkl'...")
joblib.dump(logreg_model, "logreg_fake_news_model.pkl")
print("Lưu mô hình Logistic Regression hoàn tất.")
# ------------------------------

## Mô hình 2: Support Vector Machine (LinearSVC)
print("\n--- Mô hình 2: Linear SVM ---")
svm_model = LinearSVC(class_weight='balanced')
print("Đang huấn luyện Linear SVM...")
svm_model.fit(X_train_vectorized, y_train)
print("Linear SVM huấn luyện hoàn tất.")

# --- THÊM BƯỚC LƯU MÔ HÌNH 2 ---
print("Đang lưu mô hình Linear SVM vào 'svm_fake_news_model.pkl'...")
joblib.dump(svm_model, "svm_fake_news_model.pkl")
print("Lưu mô hình Linear SVM hoàn tất.")
# ------------------------------

Đang xử lý 8741 mẫu...
Hoàn thành! Còn lại 8715 mẫu sau khi xử lý.
Đang xử lý 486 mẫu...
Hoàn thành! Còn lại 486 mẫu sau khi xử lý.

Đang tạo TF-IDF Features...
Đang lưu TF-IDF Vectorizer vào 'tfidf_vectorizer.pkl'...
Lưu Vectorizer hoàn tất.

--- Mô hình 1: Logistic Regression ---
Đang huấn luyện Logistic Regression...
Logistic Regression huấn luyện hoàn tất.
Đang lưu mô hình Logistic Regression vào 'logreg_fake_news_model.pkl'...
Lưu mô hình Logistic Regression hoàn tất.

--- Mô hình 2: Linear SVM ---
Đang huấn luyện Linear SVM...
Linear SVM huấn luyện hoàn tất.
Đang lưu mô hình Linear SVM vào 'svm_fake_news_model.pkl'...
Lưu mô hình Linear SVM hoàn tất.
